In [2]:
import numpy as np
import cv2
import pandas as pd

In [ ]:
help(cv2.threshold)

In [3]:
#Read the image
base = './'
im = cv2.imread(base + 'skyscraper.jpg')
gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
equ = cv2.equalizeHist(gray)
#ret, thresh = cv2.threshold(equ,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
winSize = 7
blured = cv2.GaussianBlur(equ, (winSize, winSize), 0)
edges = cv2.Canny(blured,100,220,apertureSize = 7, L2gradient=True)



#cv2.imshow('houghlines',im)
cv2.imwrite(base+'gray.jpg', gray)
cv2.imwrite(base+'equ.jpg', equ)
cv2.imwrite(base+'blured.jpg', blured)
cv2.imwrite(base+'edges.jpg', edges)

#cv2.imwrite(base+'thresh.jpg', thresh)

#A few changes to check git functionality

True

In [ ]:
help(cv2.Canny)

In [8]:
help(cv2.HoughLines)

Help on built-in function HoughLines:

HoughLines(...)
    HoughLines(image, rho, theta, threshold[, lines[, srn[, stn[, min_theta[, max_theta]]]]]) -> lines
    .   @brief Finds lines in a binary image using the standard Hough transform.
    .   
    .   The function implements the standard or standard multi-scale Hough transform algorithm for line
    .   detection. See <http://homepages.inf.ed.ac.uk/rbf/HIPR2/hough.htm> for a good explanation of Hough
    .   transform.
    .   
    .   @param image 8-bit, single-channel binary source image. The image may be modified by the function.
    .   @param lines Output vector of lines. Each line is represented by a two-element vector
    .   \f$(\rho, \theta)\f$ . \f$\rho\f$ is the distance from the coordinate origin \f$(0,0)\f$ (top-left corner of
    .   the image). \f$\theta\f$ is the line rotation angle in radians (
    .   \f$0 \sim \textrm{vertical line}, \pi/2 \sim \textrm{horizontal line}\f$ ).
    .   @param rho Distance resolution

In [142]:
im.shape

(757, 543, 3)

In [141]:
height = im.shape[0]
width = im.shape[1]

In [143]:
def draw_line (img, rho, theta, color=(0,255,0)):
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(np.around(x0 + 1000*(-b)))   # Here i have used int() instead of rounding the decimal value, so 3.8 --> 3
    y1 = int(np.around(y0 + 1000*(a)))    # But if you want to round the number, then use np.around() function, then 3.8 --> 4.0
    x2 = int(np.around(x0 - 1000*(-b)))   # But we need integers, so use int() function after that, ie int(np.around(x))
    y2 = int(np.around(y0 - 1000*(a)))
    cv2.line(img,(x1,y1),(x2,y2),color,2)
    print (rho, theta, color)

In [82]:
np.pi / 2

1.5707963267948966

In [150]:
img = im.copy()
rho_minus = 0
rho_plus = np.inf

rho_hor_max = 0
hor_max_found = False
rho_hor_min = im.shape[0]
hor_min_found = False
eps = 0.01
lines = cv2.HoughLines(edges,1,np.pi/180, int (min(im.shape[0], im.shape[1]) / 3))
#print (lines)
for line in lines:
    for rho,theta in line:
        if (theta > np.pi / 2 + eps and np.abs (rho) > np.abs (rho_minus)):
            rho_minus = rho
            theta_minus = theta
        if (theta < np.pi / 2 - eps and rho < rho_plus):
            rho_plus = rho
            theta_plus = theta
        if (np.abs (theta - np.pi / 2) < eps):
            if (rho > rho_hor_max):
                rho_hor_max = rho
                hor_max_found = True
            if (rho < rho_hor_min):
                rho_hor_min = rho
                hor_min_found = True
        #draw_line (img, rho, theta)
        
red = (0, 0, 255)
draw_line (img, rho_minus, theta_minus, red)
draw_line (img, rho_plus, theta_plus, red)

if hor_min_found:
    draw_line (img, rho_hor_min, np.pi / 2, red)
if hor_max_found:
    draw_line (img, rho_hor_max, np.pi / 2, red)
cv2.imwrite(base+'houghlines_red.jpg', img)
#cv2.imshow('houghlines',img)

-341.0 2.8972466 (0, 0, 255)
190.0 0.2617994 (0, 0, 255)


True

In [ ]:
img = im.copy()
lines = cv2.HoughLinesP(edges,1,np.pi/180,80, minLineLength = 100, maxLineGap = 10)
for line in lines:
    for x1,y1,x2,y2 in line:
        cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
cv2.imwrite(base+'houghlines_p.jpg', img)
cv2.imshow('houghlines',img)
cv2.waitKey()


In [107]:
# Given y, this function returns x from (rho, theta) line equation
def x_from_y(rho, theta, y):
    sin_t = np.sin(theta)
    cos_t = np.cos(theta) 
    
    return ((rho / cos_t) - ((y * sin_t) / cos_t))

In [111]:
# Given x, this function returns y from (rho, theta) line equation
def y_from_x(rho, theta, x):
    sin_t = np.sin(theta)
    cos_t = np.cos(theta) 
    
    return (-((x * cos_t) / sin_t) + (rho / sin_t)) 

In [151]:
# Given a point and the rectangle, this function checks 
# if this point lies inside the rectangle
def check_point(x, y, left_x, right_x, up_y, down_y):
    if (left_x <= x) and (x <= right_x) and (up_y <= y) and (y <= down_y):
        return (int(np.around(x)), int(np.around(y)))
    else:
        return None

In [113]:
# A line which traverses some rectangle has two points of traversal.
#
# This function finds four possible traversal points and drops those of them 
# which are not inside the rectangle
def get_traversal_points(rho, theta, left_x, right_x, up_y, down_y):      
    
    #Find four possible points of traversal
    up = (x_from_y(rho, theta, up_y), up_y)
    right = (right_x, y_from_x(rho, theta, right_x))
    down = (x_from_y(rho, theta, down_y), down_y)
    left = (left_x, y_from_x(rho, theta, left_x))
    
    #If point is inside, we add it to the list; otherwise we add None
    intersect_points = list(map(lambda p: check_point(p[0], p[1], left_x, right_x, up_y, down_y), [up, right, down, left]))
    
    #Drop all None values
    intersect_points = [point for point in intersect_points if point != None]
    
    #Check if exactly two points were found
    if len(intersect_points) != 2:
        print('Error! len(intersect_points) != 2, intersect_points = {}'.format(intersect_points))
        return None
    else:
        return sorted(intersect_points, key=lambda x: x[0])  

In [120]:
print(get_traversal_points(rho_minus, theta_minus, 0, width, 0, height))
print(get_traversal_points(rho_plus, theta_plus, 0, width, 0, height))

[(351, 0), (540, 757)]
[(42, 757), (217, 0)]


In [122]:
# Get the point of traversal for 'plus' and 'minus' lines
# The order as follows in a way to obtain convenient trapezoid
down_plus, up_plus = get_traversal_points(rho_plus, theta_plus, 0, width, 0, height)
up_minus, down_minus = get_traversal_points(rho_minus, theta_minus, 0, width, 0, height)

crop_height = min(down_minus[1], down_plus[1])
print(crop_height)

757


In [123]:
crop = im[0:crop_height, :]
cv2.imwrite(base+'cropped.jpg', crop)

True

In [125]:
# Get the points of traversal for the cropped picture
crop_down_plus, crop_up_plus = get_traversal_points(rho_plus, theta_plus, 0, width, 0, crop_height)
crop_up_minus, crop_down_minus = get_traversal_points(rho_minus, theta_minus, 0, width, 0, crop_height)

In [126]:
# Create a transform and apply it
from_points = np.float32([crop_down_plus, crop_up_plus, crop_up_minus, crop_down_minus])
to_points = np.float32([(0, crop_height), (0, 0), (width, 0), (width, crop_height)])

M = cv2.getPerspectiveTransform(from_points, to_points)
dst = cv2.warpPerspective(crop, M, (width, crop_height))

cv2.imwrite(base+'transformed.jpg', dst)

True